# RAG demo level 3 - reference with 200 movies semantic search

In [14]:
questions = [
    "What movies are about Abby?",
    "I have seen all Star Wars movies and would like tips for something similar I can watch next.",
    "What is the most common genre of the movies where one of key figures is called Mark?",
    "Are there any movies where Prague takes major role and present city as mysterious and ancient?",
    "When movies about drugs are concerned, is it usually rather serious or funny?",
    "What are some movies featuring a strong female lead that also involve adventure?",
    "Which Western films feature outlaws riding to their doom in the American Southwest?"
]

In [15]:
import subprocess
import os
import json

original_dir = os.getcwd()
try:
    # Jump into the terraform directory
    os.chdir('terraform')

    # Get the database connection string
    SQL_DATABASE = subprocess.run(['terraform', 'output', '-raw', 'SQL_DATABASE'], stdout=subprocess.PIPE).stdout.decode('utf-8')
    SQL_USERNAME = subprocess.run(['terraform', 'output', '-raw', 'SQL_USERNAME'], stdout=subprocess.PIPE).stdout.decode('utf-8')
    SQL_PASSWORD = subprocess.run(['terraform', 'output', '-raw', 'SQL_PASSWORD'], stdout=subprocess.PIPE).stdout.decode('utf-8')
    SQL_SERVER_FQDN = subprocess.run(['terraform', 'output', '-raw', 'SQL_SERVER_FQDN'], stdout=subprocess.PIPE).stdout.decode('utf-8')

    # Get the embedding model endpoint and key
    model_configurations = subprocess.run(['terraform', 'output', '-raw', 'model_configurations'], stdout=subprocess.PIPE).stdout.decode('utf-8')
    model_config = json.loads(model_configurations)
    embedding_model = model_config["models"]["text-embedding-3-large"]
    EMBEDDINGS_ENDPOINT = embedding_model["endpoint"]
    EMBEDDINGS_KEY = embedding_model["key"]
    gpt_4o_mini_model = model_config["models"]["gpt-4o-mini"]
    GPT_4O_MINI_ENDPOINT = gpt_4o_mini_model["endpoint"]
    GPT_4O_MINI_KEY = gpt_4o_mini_model["key"]
    gpt_4o_model = model_config["models"]["gpt-4o"]
    GPT_4O_ENDPOINT = gpt_4o_model["endpoint"]
    GPT_4O_KEY = gpt_4o_model["key"]

    print(f"Using {SQL_SERVER_FQDN} as the database server")
    print(f"Using {EMBEDDINGS_ENDPOINT} as the embedding model endpoint")
    print(f"Using {GPT_4O_MINI_ENDPOINT} as the gpt-4o-mini model endpoint")
    print(f"Using {GPT_4O_ENDPOINT} as the gpt-4o model endpoint")

finally:
    os.chdir(original_dir)

Using sql-graphrag-psbv.database.windows.net as the database server
Using https://graphrag-psbv.openai.azure.com/ as the embedding model endpoint
Using https://graphrag-psbv.openai.azure.com/ as the gpt-4o-mini model endpoint
Using https://graphrag-psbv.openai.azure.com/ as the gpt-4o model endpoint


In [16]:
from openai import AzureOpenAI 
import pandas as pd
import pyodbc

# Azure SQL Entra login option
# conn_str = f"DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={SQL_SERVER_FQDN};DATABASE={SQL_DATABASE};UID=tokubica@microsoft.com;Authentication=ActiveDirectoryInteractive;Encrypt=yes;"

# Azure SQL password login option
conn_str = f"DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={SQL_SERVER_FQDN};DATABASE={SQL_DATABASE};UID={SQL_USERNAME};PWD={SQL_PASSWORD};Encrypt=yes;Charset=Latin1;"

conn = pyodbc.connect(conn_str)

gpt_embedding_client = AzureOpenAI(
    azure_endpoint=EMBEDDINGS_ENDPOINT,
    api_key=EMBEDDINGS_KEY,
    api_version="2025-02-01-preview",
)

gpt_4o_client = AzureOpenAI(  
    azure_endpoint=GPT_4O_ENDPOINT,  
    api_key=GPT_4O_KEY,  
    api_version="2024-05-01-preview",
)

gpt_4o_mini_client = AzureOpenAI(
    azure_endpoint=GPT_4O_MINI_ENDPOINT,  
    api_key=GPT_4O_MINI_KEY,  
    api_version="2024-05-01-preview",
)

In [27]:
import pandas as pd
from pandas import DataFrame
from IPython.display import display, Markdown
import jinja2
import time

max_retries = 20
retry_delay = 10

def get_embeddings(texts):
    for attempt in range(1, max_retries + 1):
        try:
            result = gpt_embedding_client.embeddings.create(
                input=texts,
                model="text-embedding-3-large",
                dimensions=1998,
            )
            break
        except Exception as e:
            if hasattr(e, "status_code") and e.status_code == 429:
                time.sleep(retry_delay)
            else:
                print(f"Embedding generation error: {e}")
                raise e
    return [json.dumps(item.embedding) for item in result.data]

def print_output(df_results: list[DataFrame], question: str, response: str = "") -> None:
    print("\n\n*************************************************")
    print(f"Top matching nodes for question: {question}")
    for idx, df in enumerate(df_results):
        display(df)
    print("LLM response:")
    display(Markdown(response))

def llm_final_answer(question: str, context_jsons: list[str]) -> str:
    with open("./prompts/query.jinja2", 'r') as f:
        system_prompt_template = jinja2.Template(f.read())
    system_prompt = system_prompt_template.render()
    messages = [{"role": "system", "content": system_prompt}]
    for context_json in context_jsons:
        messages.append({"role": "user", "content": context_json})
    messages.append({"role": "user", "content": question})
    
    for attempt in range(1, max_retries + 1):
        try:
            completion = gpt_4o_client.chat.completions.create(
                model="gpt-4o",
                messages=messages,
                max_tokens=1000,
                temperature=0.7,
            )
            break
        except Exception as e:
            if hasattr(e, "status_code") and e.status_code == 429:
                print(f"LLM response rate limited. Retrying in {retry_delay} seconds (Attempt {attempt}/{max_retries})")
                time.sleep(retry_delay)
            else:
                print(f"LLM response error: {e}")
                raise e
    else:
        raise Exception("Failed to get LLM final answer after maximum retries.")
    
    return completion.choices[0].message.content

def similarity_search(conn, query: str, top_k: int = 5):
    query_embedding = get_embeddings([query])[0]
    cursor = conn.cursor()
    command = f"""
    DECLARE @q VECTOR(1998) = '{query_embedding}';
    SELECT TOP {top_k} Id, Content, vector_distance('cosine', Embedding, @q) AS Similarity
    FROM [dbo].[Movie]
    WHERE Embedding IS NOT NULL AND Content IS NOT NULL
    ORDER BY Similarity ASC
    """

    cursor.execute(command)
    rows = cursor.fetchall()

    columns = [column[0] for column in cursor.description]
    result = [dict(zip(columns, row)) for row in rows]
    
    return result

In [28]:
for question in questions:
    context_json = json.dumps(similarity_search(conn, question, top_k=200))
    answer = llm_final_answer(question, [context_json])
    print_output([pd.DataFrame(json.loads(context_json))], question, answer)
    



*************************************************
Top matching nodes for question: What movies are about Abby?


,Id,Content,Similarity
0,11082,TITLE: The Seventh Sign OVERVIEW: Abby is a pr...,0.500736
1,78383,"TITLE: Nurse 3-D OVERVIEW: Abby Russell, a bea...",0.552392
2,40205,TITLE: 16 Wishes OVERVIEW: The story about Abb...,0.584054
3,2005,TITLE: Sister Act OVERVIEW: A Reno singer witn...,0.593369
4,362057,TITLE: Martyrs OVERVIEW: A woman and her child...,0.598343
...,...,...,...
195,416051,TITLE: Based on a True Story OVERVIEW: Delphin...,0.659827
196,13824,TITLE: Superstar OVERVIEW: Orphan Mary Katheri...,0.659923
197,396810,TITLE: The Last Word OVERVIEW: A retired busin...,0.659989
198,146229,TITLE: Kristy OVERVIEW: When a college girl wh...,0.659991


LLM response:


Here are the movies featuring characters named Abby:

1. **The Seventh Sign**  
   - **Overview**: Abby is a pregnant woman who has a mysterious boarder living in the apartment above her garage. This boarder turns out to be heaven-sent and is accelerating the apocalypse by following scripture, causing rivers to run bloody and plagues to emerge.

2. **Nurse 3-D**  
   - **Overview**: Abby Russell is a beautiful yet sinister nurse with a dark secret. She leads a double life, targeting and punishing men who are dishonest.

3. **16 Wishes**  
   - **Overview**: Abby Jensen is a teenager who has been eagerly waiting for her 16th birthday and has kept a secret wish list since she was a little girl. On her big day, disaster strikes, but magical birthday candles help her wishes come true. However, one wish threatens to change everything.



*************************************************
Top matching nodes for question: I have seen all Star Wars movies and would like tips for something similar I can watch next.


...

Here are some movies featuring strong female leads in adventure-driven narratives:

1. **Romancing the Stone** (1984): Novelist Joan Wilder finds herself on a thrilling adventure in South America to rescue her abducted sister after receiving a mysterious package. Her journey takes her deep into the jungle, where she faces unexpected challenges.

2. **Lara Croft: Tomb Raider** (2001): Lara Croft, an English aristocrat and archaeologist, takes on a secret society while recovering lost antiquities and battling the villain Powell, who seeks a powerful relic.

3. **Tomb Raider** (2018): Lara Croft, the daughter of a missing adventurer, embarks on an adventure to a mysterious island to uncover the truth behind her father's disappearance.

4. **Wonder Woman** (2017): Amazonian princess Diana defies her people’s rules and ventures into the human world during World War I to confront evil and bring peace.

5. **Brave** (2012): Princess Mérida, an accomplished archer, defies sacred customs in the Scottish Highlands, setting off on a unique adventure that challenges her courage and resourcefulness.

6. **EverAfter** (1998): Set in 16th-century France, Danielle, a spirited woman with a love for books, embarks on a journey that intertwines romance and her quest for justice.

7. **The River Wild** (1994): Gail, a rafting expert, uses her skills to confront armed killers while navigating a violent river during a family vacation.

8. **Wild** (2014): A woman with a tragic past decides to hike a thousand miles on the Pacific Crest Trail to rediscover herself and find a new meaning in life.

9. **Tracks** (2013): An Australian woman undertakes a solo journey across the desert, accompanied only by her dog and camels, in search of solitude and self-discovery.

10. **Cutthroat Island** (1995): Morgan Adams, a female pirate, leads her crew on a perilous quest to recover pieces of a treasure map while battling familial betrayal and external threats.

These films showcase strong female protagonists who overcome challenges, embark on daring journeys, and redefine their destinies.



*************************************************
Top matching nodes for question: Which Western films feature outlaws riding to their doom in the American Southwest?


,Id,Content,Similarity
0,576,"TITLE: The Wild Bunch OVERVIEW: Aging outlaw, ...",0.438358
1,10747,TITLE: The Outlaw Josey Wales OVERVIEW: After ...,0.463956
2,642,TITLE: Butch Cassidy and the Sundance Kid OVER...,0.478436
3,335,TITLE: Once Upon a Time in the West OVERVIEW: ...,0.485921
4,3088,TITLE: My Darling Clementine OVERVIEW: Wyatt E...,0.490831
...,...,...,...
195,2099,TITLE: Raw Deal OVERVIEW: Mark Kaminsky is kic...,0.660070
196,227,TITLE: The Outsiders OVERVIEW: When two poor G...,0.660287
197,15417,TITLE: Every Which Way But Loose OVERVIEW: Phi...,0.660309
198,4476,TITLE: Legends of the Fall OVERVIEW: An epic t...,0.660623


LLM response:


Several Western films feature outlaws riding to their doom in the American Southwest, often showcasing themes of betrayal, revenge, or redemption. Here are a few notable examples:

1. **The Wild Bunch**: Aging outlaw Pike Bishop plans to retire after one last robbery, but the heist turns out to be a setup by his former partner, Deke Thornton. As the gang escapes into Mexican territory, they face fierce gunfights and high casualties, ultimately riding toward their grim fate.

2. **Butch Cassidy and the Sundance Kid**: Set in the late 1890s in Wyoming, Butch Cassidy and the Sundance Kid lead a gang of outlaws. Pursued relentlessly by lawmen, they flee to South America in a desperate attempt to escape, ultimately facing their doom as the world modernizes around them.

3. **The Assassination of Jesse James by the Coward Robert Ford**: Outlaw Jesse James, notorious for being the "fastest gun in the West," meets his end when jealousy and betrayal by Robert Ford lead to his assassination, solidifying his legendary status in the American Southwest.

4. **3:10 to Yuma**: Both versions of this film depict infamous outlaw Ben Wade being transported to the "3:10 to Yuma" train for trial. His gang attempts to rescue him, leading to a tense and ultimately tragic showdown.

5. **Unforgiven**: Retired gunslinger William Munny takes on one last bounty-hunting mission to track down men who brutalized a prostitute. Joined by his former partner, Munny rides into a violent and unforgiving confrontation, showcasing the harsh realities of the Southwest.

6. **High Plains Drifter**: A mysterious gunfighter arrives in the settlement of Lago, where he is hired to protect the town from three outlaws. The stranger’s past and vengeance intertwine as doom looms for all involved.

These films offer dramatic portrayals of outlaws navigating the rugged landscapes and moral complexities of the American Southwest, often culminating in their inevitable downfall.